In [1]:
api_key = "hh8DW29tdDmshbghN71NJPI0jgFzp1Ay5MzjsniIz9a8Piq7xp"

### Making the API request for all cards

In [2]:
import requests
headers = {'X-Mashape-Key': api_key}

In [3]:
# cardname = "Leeroy"

In [4]:
# endpoint_one_card = f"https://omgvamp-hearthstone-v1.p.mashape.com/cards/search/{cardname}"
# endpoint_allcards = "https://omgvamp-hearthstone-v1.p.mashape.com/cards"

def get_cards_by_type(card_type, collectible=1, cost=None, durability=None, health=None, key=api_key):
    endpoint_by_type = f'https://omgvamp-hearthstone-v1.p.mashape.com/cards/types/{card_type}'
    payload = {'collectible': collectible, 'cost': cost, 'durability': durability, 'health': health}
    r = requests.get(endpoint_by_type, params=payload, headers=headers)
    return r.json()

In [5]:
# r = requests.get(endpoint_allcards, headers=headers)

In [6]:
# r.json()

In [7]:
import json
# with open('hs_data.json', 'w') as outfile:
#     json.dump(r.json(), outfile)

### Using the offline JSON

In [8]:
import pandas as pd

In [33]:
# Reading the json as a dict
with open('hs_data.json') as json_data:
    data = json.load(json_data)

In [37]:
everything = [single_card for cardset in data.values() for single_card in cardset]

In [39]:
collectibles = [single_card for single_card in everything 
                if 'collectible' in single_card 
                and single_card['collectible']]

In [41]:
non_heroes = [single_card for single_card in collectibles
              if single_card['type'] != 'Hero']

In [53]:
all_types = set([card['type'] for card in non_heroes])
all_types

{'Minion', 'Spell', 'Weapon'}

### Otherwise, first separate the cards by type. We can make API calls for that.

In [12]:
# collectible_spells = get_cards_by_type('Spell')
# collectible_weapons = get_cards_by_type('Weapon')
# collectible_minions = get_cards_by_type('Minion')

In [13]:
# for cardtype, collection in zip(['minions', 'spells', 'weapons'], [collectible_minions, collectible_spells, collectible_weapons]):
#     with open(f'{cardtype}.json', 'w') as outfile:
#         json.dump(collection, outfile)

In [14]:
# Reading the json as a dict
with open('minions.json') as json_data:
    minions = json.load(json_data)
    
with open('spells.json') as json_data:
    spells = json.load(json_data)
    
with open('weapons.json') as json_data:
    weapons = json.load(json_data)

### Let's now separate the titles, flavors, texts, mechanics, costs and stats.
#### Each card type has different attributes and design logic behind them, so we want to make educated splits. Moreover, not all fields are of interest for us for now. 

In [15]:
spell_attributes = {"name", "cardId", "cost", "img", "playerClass", "rarity", "text", "flavor", "mechanics"}
# spell_optional_attributes = {}

minion_attributes = {"name", "cardId", "cost", "health", "attack", "img", "playerClass", "rarity", "text", "flavor", "mechanics"}
# minion_optional_attributes = {}

weapon_attributes = {"name", "cardId", "cost", "durability", "attack", "img", "playerClass", "rarity", "text", "flavor", "mechanics"}
# weapon_attributes = {}

### For this first generation example we shall funnel all the cards to the above attributes to normalize the data.

In [16]:
def normalize_card(card, attrs):
    
    concise = {a: card[a] if a in card else None for a in attrs}
    
    return concise

In [18]:
spells_concise = [normalize_card(spell_card, spell_attributes) for spell_card in spells]
minions_concise = [normalize_card(minion_card, minion_attributes) for minion_card in minions]
weapons_concise = [normalize_card(weapon_card, weapon_attributes) for weapon_card in weapons]

In [19]:
spells_concise[:2]

[{'cardId': 'CS2_041',
  'cost': 0,
  'flavor': 'I personally prefer some non-ancestral right-the-heck-now healing, but maybe that is just me.',
  'img': 'http://media.services.zam.com/v1/media/byName/hs/cards/enus/CS2_041.png',
  'mechanics': [{'name': 'Taunt'}],
  'name': 'Ancestral Healing',
  'playerClass': 'Shaman',
  'rarity': 'Free',
  'text': 'Restore a minion\\nto full Health and\\ngive it <b>Taunt</b>.'},
 {'cardId': 'CS2_072',
  'cost': 0,
  'flavor': 'It\'s funny how often yelling "Look over there!" gets your opponent to turn around.',
  'img': 'http://media.services.zam.com/v1/media/byName/hs/cards/enus/CS2_072.png',
  'mechanics': None,
  'name': 'Backstab',
  'playerClass': 'Rogue',
  'rarity': 'Free',
  'text': 'Deal $2 damage to an undamaged minion.'}]

In [1]:
# titles = [(card['name'], card['cardId']) for card in non_heroes]
# flavors = [(card['flavor'], card['cardId']) for card in non_heroes]
# texts = [(card['text'], card['cardId']) for card in non_heroes]
# mechanics = [(card['mechanics'], card['cardId']) if 'mechanics' in card else None for card in non_heroes]
# costs = [(card['cost'], card['cardId']) for card in non_heroes]
# stats = [(card['name'], card['cardId']) for card in non_heroes]

### The data is now quite ready to be modelled! We can now access the needed fields directly from the correpsonding cardtype set.

## A small LSTM

In [38]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

### We will first try to generate card flavors, as those resemble normal text the most. For the flavors we can combine all the flavors from all types of cards, as they are mostly just humorous pieces of text and are not directly associated with the type.

In [21]:
def get_all_flavortexts():
    
    weapon_flavors = [card['flavor'] for card in weapons]
    minion_flavors = [card['flavor'] for card in minions]
    spell_flavors = [card['flavor'] for card in spells if 'flavor' in card]
    
    return weapon_flavors + minion_flavors + spell_flavors

In [22]:
all_flavors = get_all_flavortexts()

In [37]:
all_flavors[73]

'Clockwork gnomes are always asking what time it is.'

### Our average sequence length is 67.8 characters. This is handy to know for the LSTM sequence length parameter. Let's round it down for sake of memorability.

In [40]:
average_sequence_length = np.mean([len(list(flavor)) for flavor in all_flavors])
average_sequence_length

67.86418193303854

In [42]:
SEQUENCE_LENGTH = 65

### Our generative model is a character based one, so our input data is a huge list of characters.

In [26]:
all_flavor_chars = [ch for one_sentence in all_flavors for ch in list(one_sentence)]

In [29]:
all_flavor_unique_chars = set(all_flavor_chars)

### We see that there are a lot of unnecessary or unwished characters in the model. We could clean up the model by moderating the char list but for now we let it like this.

In [30]:
print(f'Data length: {len(all_flavor_chars)} characters')
print(f'Vocabulary size: {len(all_flavor_unique_chars)} characters')

Data length: 107429 characters
Vocabulary size: 94 characters


### The unique characters are the features for our model. Let's numerify them to make them ML ready.

In [31]:
ix_to_char = {ix:char for ix, char in enumerate(all_flavor_unique_chars)}
char_to_ix = {char:ix for ix, char in enumerate(all_flavor_unique_chars)}